In [ ]:
# import libraries
!pip install tensorflow
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 47.5 MB/s eta 0:00:00


In [ ]:
# import libaries
import io
import re
import string
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
import os
import torch
#import pacakges
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

In [ ]:
!pip install pydub

In [ ]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding, Input, TimeDistributed, GlobalAveragePooling2D, ConvLSTM2D, Reshape, Concatenate

In [ ]:
# Build the audio model
import tensorflow as tf
from transformers import TFAutoModel

# Load the pre-trained model using the Hugging Face interface
model_checkpoint = "facebook/wav2vec2-base"
huggingface_model = TFAutoModel.from_pretrained(model_checkpoint, trainable=False, from_pt=True)

# Define the inputs to the model
input_values = tf.keras.Input(shape=(16000,), dtype=tf.float32)

# Pass the inputs through the Wav2Vec model
wav2vec_output = huggingface_model(input_values)


# Create the TensorFlow functional API model
audio_model = tf.keras.Model(inputs=input_values, outputs=wav2vec_output)

# Print the model summary
audio_model.summary()

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(



TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tine this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['project_hid.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFWav2Vec2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the che

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16000)]           0         
                                                                 
 tf_wav2_vec2_model (TFWav2V  TFWav2Vec2BaseModelOutpu  94371712 
 ec2Model)                   t(last_hidden_state=(Non            
                             e, 49, 768),                        
                              extract_features=(None,            
                              49, 512),                          
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
Total params: 94,371,712
Trainable params: 0
Non-trainable params: 94,371,712
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense
from tensorflow.keras import layers

In [ ]:
# Time model
# Define input layers
time_stamps = Input(shape=(50, 2))

# Define embedding layer

# Embed word and pos inputs
#word_embedded = embedding_layer(word_input)


# Concatenate word and pos embeddings
#concatenated = Concatenate()([word_embedded, time_stamps])

# Reshape the concatenated tensor
#reshaped = Reshape((-1, 16))(concatenated)

# Apply LSTM layer
lstm_output = LSTM(16, dropout=0.2, recurrent_dropout=0.2)(time_stamps)

# Apply dense layer for binary classification
#output = Dense(1, activation='sigmoid')(lstm_output)

# Define the model
time_model = Model(inputs=time_stamps, outputs=lstm_output, name='time_model')

# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
time_model.summary()

Model: "time_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 2)]           0         
                                                                 
 lstm (LSTM)                 (None, 16)                1216      
                                                                 
Total params: 1,216
Trainable params: 1,216
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Concatenante audio and time models
from tensorflow.keras.layers import Reshape, RepeatVector, Concatenate
# Reshape word_model output to match the shape of audio_model output
audio_model_output = layers.GlobalAveragePooling1D()(audio_model.output[1])
# Drop-out layer before the final Classification-Head
audio_model_output = layers.Dropout(0.5) (audio_model_output)

concatenated_output = Concatenate()([audio_model_output, time_model.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
combined= Dense(1, activation='sigmoid')(concatenated_output)
audio_time_model = Model(inputs=[audio_model.input, time_model.input], outputs=combined, name='audio_time_model')
audio_time_model.summary()

Model: "audio_time_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_1[0][0]']                
 Model)                         Output(last_hidden_                                               
                                state=(None, 49, 76                                               
                                8),                                                               
                                 extract_features=(                                               
                                None, 49, 512),                                    

In [ ]:
# Load and split the dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import load_from_disk
final_combined_dataset=load_from_disk('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented')
final_combined_dataset=final_combined_dataset.train_test_split(test_size=0.2)
final_trained_dataset=final_combined_dataset['train'].train_test_split(test_size=0.2)
# Convert nested lists to numpy arrays
audio_train = pad_sequences(final_trained_dataset['train']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_train = np.asarray(audio_train).astype(np.float32)
#word_train = np.asarray(final_combined_dataset['train']['word']).astype(np.float32)
time_train = np.asarray(final_trained_dataset['train']['time_stamps']).astype(np.float32)
audio_val = pad_sequences(final_trained_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_val = np.asarray(audio_val).astype(np.float32)
audio_test = pad_sequences(final_combined_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_test = np.asarray(audio_test).astype(np.float32)
#word_test = np.asarray(final_combined_dataset['test']['word']).astype(np.float32)
time_val = np.asarray(final_trained_dataset['test']['time_stamps']).astype(np.float32)
time_test = np.asarray(final_combined_dataset['test']['time_stamps']).astype(np.float32)
y_train=np.asarray(final_trained_dataset['train']['label']).astype(np.float32)
y_val=np.asarray(final_trained_dataset['test']['label']).astype(np.float32)
y_test=np.asarray(final_combined_dataset['test']['label']).astype(np.float32)

In [ ]:
# Compile and train the model
audio_time_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)
audio_time_model.fit([audio_train, time_train], y_train,
                       epochs=50, batch_size=16, validation_data=([audio_val, time_val], y_val),
                       callbacks=callback)

Epoch 1/50
2158/2158 [==============================] - 407s 182ms/step - loss: 0.6732 - accuracy: 0.5794 - precision: 0.5016 - recall: 0.2573 - auc: 0.5774 - val_loss: 0.6534 - val_accuracy: 0.5988 - val_precision: 0.5555 - val_recall: 0.2811 - val_auc: 0.6325
Epoch 2/50
2158/2158 [==============================] - 385s 178ms/step - loss: 0.6551 - accuracy: 0.6063 - precision: 0.5505 - recall: 0.3562 - auc: 0.6291 - val_loss: 0.6503 - val_accuracy: 0.6117 - val_precision: 0.5532 - val_recall: 0.4490 - val_auc: 0.6405
Epoch 3/50
2158/2158 [==============================] - 384s 178ms/step - loss: 0.6533 - accuracy: 0.6043 - precision: 0.5457 - recall: 0.3626 - auc: 0.6309 - val_loss: 0.6515 - val_accuracy: 0.6079 - val_precision: 0.5751 - val_recall: 0.2963 - val_auc: 0.6384
Epoch 4/50
2158/2158 [==============================] - 383s 178ms/step - loss: 0.6506 - accuracy: 0.6102 - precision: 0.5547 - recall: 0.3780 - auc: 0.6356 - val_loss: 0.6432 - val_accuracy: 0.6158 - val_precision

In [ ]:
# Evaluate the model
audio_time_model.evaluate([audio_test, time_test], y_test)

338/338 [==============================] - 63s 183ms/step - loss: 0.6335 - accuracy: 0.6277 - precision: 0.5940 - recall: 0.3113 - auc: 0.6668


[0.6334820985794067,
 0.6277345418930054,
 0.5940042734146118,
 0.31133556365966797,
 0.6668455600738525]